# Data

## Crowd human

In [2]:
from pathlib import Path

data_dir = Path("/home/jasseur/Downloads/crowdhuman")

In [3]:
GENDERS = ["M", "F"]
AGES = ["1-2", "3-9", "10-20", "21-25", "26-27", "28-31", "32-36", "37-45", "46-54", "55-65", "66-116"]

In [ ]:
import pandas as pd

def read_df(data_file):
    data_df = pd.read_csv(data_file)
    data_df = data_df[data_df["gender"].notnull() & data_df["age"].notnull()]
    data_df = data_df.reset_index(drop=True)
    return data_df

In [ ]:
import pandas as pd

train_crowd_file = Path("../../../data/crowdhuman/annotation_train_with_classes.csv")
train_df = read_df(train_crowd_file)
train_df.head(1)

In [ ]:
val_crowd_file = Path("../../../data/crowdhuman/annotation_val_with_classes.csv")
val_df = read_df(val_crowd_file)
val_df.head(1)

In [ ]:
import random
import albumentations as A
import albumentations.augmentations.geometric.functional as F
import cv2

class AspectRatioResize(A.DualTransform):
    """Resize the input to the given height and width.

    Args:
        p (float): probability of applying the transform. Default: 1.
        height (int): desired height of the output.
        width (int): desired width of the output.
        interpolation (OpenCV flag): flag that is used to specify the interpolation algorithm. Should be one of:
            cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4.
            Default: cv2.INTER_LINEAR.

    Targets:
        image, mask, bboxes

    Image types:
        uint8, float32
    """

    def __init__(
        self, height, width, scale=(1, 1), interpolation=cv2.INTER_LINEAR, always_apply=False, p=1
    ):
        super(AspectRatioResize, self).__init__(always_apply, p)
        self.height = height
        self.width = width
        self.scale = scale
        self.interpolation = interpolation

    def apply(self, img, interpolation=cv2.INTER_LINEAR, **params):
        height, width, _ = img.shape
        r = min(self.width / width, self.height / height)
        scale = random.uniform(*self.scale)
        return F.resize(
            img, height=int(scale * r * height), width=int(scale * r * width),
            interpolation=interpolation
        )

    def apply_to_bbox(self, bbox, **params):
        # Bounding box coordinates are scale invariant
        return bbox

In [ ]:
import albumentations as A

IMAGE_SIZE = (1088, 608)

test_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], scale=(1, 1), p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=[]
    )
)

In [ ]:
from src.test_dataset import TestDataset
%load_ext autoreload
%autoreload 2

train_dir = Path(data_dir, "Images")
test_dataset = TestDataset(train_dir, val_df, test_transform)

In [ ]:
# test_dataset.data_groups = test_dataset.data_groups[:1000]

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

image_name, bboxes_id, image, centers = test_dataset[16].values()
for bbox_id, center in zip(bboxes_id, centers):
    x, y = map(int, center)
    cv2.circle(
        image, (x, y), 20, (255, 0, 0), 2, lineType=cv2.LINE_AA
    )

plt.figure()
plt.imshow(image)
plt.show()

# Prediction

In [ ]:
import sys
sys.path.append("../..")
from person_detection.centernet.src.pose_dla_dcn import get_pose_net as get_dla_dcn
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

num_layers = 34
heads = {"hm": 1, "wh": 4, "reg": 2, "id": 128, "gender": len(GENDERS), "age": len(AGES)}
head_conv = 256
net = get_dla_dcn(num_layers, heads, head_conv)
model_path = Path("checkpoint/10.pth")
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda:1", checkpoint_dir="checkpoint", model_path=model_path
)

In [ ]:
from src.test_dataset import TestDataset
%load_ext autoreload
%autoreload 2

test_dataset = TestDataset(train_dir, train_df, test_transform)

In [ ]:
test_df = trainer.predict(test_dataset, batch_size=16)

In [ ]:
test_df.head()